<a href="https://colab.research.google.com/github/socrisogo/bc_social_media_productivity/blob/main/Proyecto_TTECH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Presentación

Este proyecto se enfoca en el análisis exploratorio, descriptivo y de negocio sobre el mercado laboral global en el área de inteligencia artificial y aprendizaje automático, utilizando un conjunto de datos que contiene ofertas de empleo en diferentes países. A través de un enfoque técnico dividido en varias fases (ingestión, limpieza, análisis descriptivo y análisis de negocio), se busca extraer conocimientos clave sobre salarios, demanda de habilidades, adopción del trabajo remoto y patrones geográficos.

# Justificación

Este proyecto se estructura en cuatro fases técnicas clave: ingestión de datos, preparación y limpieza, análisis descriptivo y análisis de negocio. A partir de un conjunto de datos  que simula más de 15.000 ofertas de empleo relacionadas con inteligencia artificial en más de 50 países, se trabajará en la carga eficiente de los datos, validación de su estructura y conversión de tipos. Posteriormente, se realizará la limpieza de valores nulos o inconsistentes, la normalización de variables como monedas, niveles de experiencia y categorías de trabajo, así como la transformación de fechas y estandarización de ubicaciones. En la etapa de análisis descriptivo se aplicarán técnicas estadísticas y visualizaciones para explorar distribuciones salariales, demanda de habilidades, modalidades de empleo y tendencias temporales. Finalmente, en el análisis de negocio se evaluarán variaciones salariales según tamaño de empresa, tipo de contrato, país y relación remota, además del impacto de las habilidades requeridas y la educación mínima. Este enfoque técnico permitirá generar conclusiones relevantes para modelos predictivos, inteligencia empresarial, planificación profesional y análisis de brechas laborales en el mercado global de IA para 2025.

# Preguntas de negocio

1. Compensación y salarios

¿Qué países ofrecen los salarios promedio más altos en el área de IA? 

¿Cómo varía el salario promedio según el nivel de experiencia (EN(Junior), MI(Intermediate), SE(Senior), EX(Director))?

¿Cuál es la relación entre el tamaño de la empresa (S(Small), M(Medium), L(Large)) y el salario ofrecido?

¿Qué categorías de trabajo (científico de datos, ingeniero de ML, etc.) tienen mayores salarios?

¿Existe diferencia salarial significativa entre tipos de empleo (FT(Full time), PT(Part time), CT(Contract), FL(Freelance))?

¿Cuál es la relación entre el porcentaje de trabajo remoto y el salario promedio?

¿Qué industrias pagan más para roles de IA?

2. Análisis geográfico

¿Qué países concentran la mayor cantidad de ofertas laborales en IA?

¿Cómo se distribuyen los tipos de contratos por país?

¿Cuál es la relación entre ubicación de la empresa y residencia del empleado?

¿Dónde es más frecuente el trabajo remoto o híbrido?

¿Qué países tienen mayor demanda de determinadas habilidades?


3. Tendencias y evolución del mercado

¿Cómo ha evolucionado la oferta de empleo en IA a lo largo del tiempo?

¿Que habilidades son mas demandadas?

¿Se han incrementado las ofertas con modalidad remota?

¿Cuáles son los picos de publicación de vacantes en el año?

¿Se observan patrones en la duración entre la fecha de publicación y la fecha límite?

# Ingestión de datos

In [1]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from scipy.stats import kurtosis
from scipy.stats import pearsonr
from scipy.stats import ttest_ind

#socialmedia = pd.read_csv(r"C:\Users\scsoto\Documents\Analitica de Datos\social_media_vs_productivity.csv")
iajobs = pd.read_csv(r"C:\Users\scsoto\Documents\Analitica de Datos\diversified_job_postings.csv")
iajobs.head(10)
#socialmedia.info()
#socialmedia.describe(include="all")


Matplotlib is building the font cache; this may take a moment.


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\scsoto\\Documents\\Analitica de Datos\\diversified_job_postings.csv'

# Preparación y limpieza

In [ ]:

#Reemplzar valores de nivel de experiencia 
iajobs['experience_level']=iajobs['experience_level'].str.strip().replace({"SE":"Expert","MI":"Intermediate","EN":"Junior","EX":"Director"})
iajobs['experience_level'].value_counts()
#iajobs = iajobs.drop('job_description_length', axis=1)
#iajobs.columns

#Reemplzar valores de tipo de empleo
iajobs['employment_type']=iajobs['employment_type'].str.strip().replace({"PT":"Part-time","FT":"Full-time","CT":"Contract","FL":"Freelance"})
iajobs['employment_type'].value_counts()

#Reemplzar valores de tamaño de compañia
iajobs['company_size']=iajobs['company_size'].str.strip().replace({"S":"Small","M":"Medium","L":"Large"})
iajobs['company_size'].value_counts()


# Analítica descriptiva

In [ ]:
# Agrupamos por company name y sacamos la media de los salarios y des score de los beneficios
df_grouped = iajobs.groupby("experience_level")[["salary_usd","benefits_score"]].mean().round(2)
print(df_grouped.to_string(float_format='{:,.2f}'.format))
#df_grouped

fig, ax1 = plt.subplots(figsize=(12, 6))

# Eje Y1: salarios
ax1.bar(df_grouped.index, df_grouped['salary_usd'], color='skyblue', label='Salary USD')
ax1.set_ylabel('Salary (USD)', color='skyblue')
ax1.tick_params(axis='y', labelcolor='skyblue')
ax1.tick_params(axis='x', rotation=90)

# Eje Y2: benefits_score
ax2 = ax1.twinx()
ax2.plot(df_grouped.index, df_grouped['benefits_score'], color='orange', marker='o', label='Benefits Score')
ax2.set_ylabel('Benefits Score', color='orange')
ax2.tick_params(axis='y', labelcolor='orange')

plt.title('Salary vs Benefits Score by Experience Level')
plt.tight_layout()
plt.show()

df_grouped = iajobs.groupby(["job_title","company_name"])[["salary_usd", "benefits_score"]].mean()
print(df_grouped.to_string(float_format='{:,.2f}'.format))

# Datos base
df = df_grouped.reset_index()

# Crear grid
g = sns.FacetGrid(df, col="company_name", col_wrap=3, height=4, sharex=False)

# Función personalizada que dibuja barra + línea
def dual_axis_plot(data, color, **kwargs):
    ax = plt.gca()

    # Barras: salario
    ax.bar(data['job_title'], data['salary_usd'], color='skyblue')
    ax.set_ylabel('Salary USD', color='skyblue')
    ax.tick_params(axis='y', labelcolor='skyblue')
    ax.tick_params(axis='x', rotation=90)

    # Eje secundario: benefits
    ax2 = ax.twinx()
    ax2.plot(data['job_title'], data['benefits_score'], color='orange', marker='o')
    ax2.set_ylabel('Benefits Score', color='orange')
    ax2.tick_params(axis='y', labelcolor='orange')

# Aplicar la función a cada panel
g.map_dataframe(dual_axis_plot)

# Títulos y espaciado
g.set_titles("{col_name}")
g.fig.subplots_adjust(top=0.92)
g.fig.suptitle("Salary vs Benefits Score by Job Title in Each Company")
plt.tight_layout()
plt.show()

#Validar las habilidades mas solicitadas en los diferentes trabajos
# 1. Separar los skills individuales
all_skills = iajobs['required_skills'].dropna().str.split(', ')

# 2. Aplanar la lista de listas
flat_skills = [skill.strip() for sublist in all_skills for skill in sublist]

# 3. Contar la frecuencia de cada skill
from collections import Counter
skill_counts = Counter(flat_skills)

# 4. Convertir a DataFrame para visualizar
skill_counts_df = pd.DataFrame(skill_counts.items(), columns=['skill', 'count']).sort_values(by='count', ascending=False)

print(skill_counts_df)

top_skills = skill_counts_df.head(10)  

plt.figure(figsize=(5, 5))
plt.pie(
    top_skills['count'],
    labels=top_skills['skill'],
    autopct='%1.1f%%',
    startangle=140,
    colors=plt.cm.Paired.colors
)
plt.title('Top 10 Most Requested Skills')
plt.axis('equal')  # para que sea un círculo perfecto
plt.tight_layout()
plt.show()

# Análisis de negocio